Predicción de la evolución de la diabetes en pacientes usando MLP
===

Se desea construir um modelo de regresión no lineal (redes neuronales artificiales) que permita pronósticar el progreso de la diabetes con un horizonte de doce meses con base en variables físicas y pruebas de laboratorio. Véase https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html

En este problema se tiene una base de datos de diez variables base (edad, sexo, índice de masa corporal, presión arterial, y seis variables medidas en sangre) para 442 pacientes, y un índice que mide el progreso de la diabetes un año después de la prueba. La columna Y es la variable explicada.

In [2]:
#
# La muestra se encuentra dividida en tres partes:
#
#   * X_train, y_true_train: es la muestra para estimar los parametros optimos
#
#   * X_test, y_true_test: es la muestra para seleccionar la mejor configuración
#
#   * X_val, y_true_val: es la muestra para probar el modelo en productivo
#
import warnings

import pandas as pd
import pytest
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector

import numpy as np

from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split


warnings.filterwarnings("ignore")

df = pd.read_csv("https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/diabetes.csv")

column_trans = ColumnTransformer(
    [
     ('minmaxscaler', MinMaxScaler(feature_range=(0, 0.5)), make_column_selector(dtype_include=np.number)),
    ],
    remainder = 'drop',
)

X_trans = column_trans.fit_transform(df)

names_col = list(df.columns)

data = pd.DataFrame(X_trans, columns = names_col)

y = data.pop('target')
X = data.copy()

X_train, X_test, y_true_train, y_true_test = train_test_split(
    X,
    y,
    test_size=0.30,
    random_state=170,
)

X_test, X_val, y_true_test, y_true_val = train_test_split(
    X,
    y,
    test_size=0.50,
    random_state=170,
)

from sklearn.model_selection import GridSearchCV

estimators = [("m", MLPRegressor(max_iter=100, learning_rate='adaptive',learning_rate_init=0.001,
                                 activation='identity', solver='adam'))]

pipeline = Pipeline(
    steps=estimators,
    verbose=False
)

param_grid = [
              {
                  'm__hidden_layer_sizes': [(1,),(2,),(3,),(4,),(5,)],
                  'm__random_state':[1000,1001,1002,1003,1004,1005],
                  'm__activation': ['identity', 'logistic','relu'],   #,'tanh'
                  'm__solver': ['sgd', 'adam'],  # , 'lbfgs'
               }          
]

grid = GridSearchCV(
    estimator = pipeline,
    param_grid = param_grid,
    cv = 5
    )

grid.fit(X_train, y_true_train)

y_pred_test = grid.predict(X_test)

mse_val = mean_squared_error(y_true_test, y_pred_test)


In [3]:
#
# Use la muestra (X_train, y_true_train) para la estimación
# de los pesos óptimos de la red neuronal.
#
# Seleccione el modelo óptimo como aquel que minimice el error
# cuadrático medio para la muestra (X_test, y_true_test).
#
# Considere únicamente modelos desde una (1) hasta (5) 
# neuronas en la capa oculta. Considere únicamente las
# siguientes semillas para inicializar la red neuronal
# 1000, 1001, 1002, 1003, 1004, 1005.
#
# Compute el error cuadrático medio para la muestra
# (X_val, y_true_val). Esta muestra representa la operación
# del modelo en productio
# 
# Rta/
# True
#

# >>> Inserte su codigo aquí >>>

# <<<

# ---->>> Evaluación ---->>>
pytest.approx(mse_val, 0.0001) == 0.009535

False